In [1]:
!pip install langchain google-generativeai python-dotenv langchain_community langchain-google-genai faiss-cpu langchain langchain-community sentence-transformers InstructorEmbedding -q

ERROR: Exception:
Traceback (most recent call last):
  File "E:\Programacion\JD\Piragua\GeminiChat\venv310\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "E:\Programacion\JD\Piragua\GeminiChat\venv310\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "E:\Programacion\JD\Piragua\GeminiChat\venv310\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 464, in read
    s = self.fp.read(amt)
  File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\ssl.py", line 1273, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\USUARIO\AppData\Local\Programs\Python

In [9]:
import os #trabajar arhivoc de texto
from dotenv import load_dotenv
from langchain.document_loaders import TextLoader # Cargar archivo de texto
from langchain.text_splitter import CharacterTextSplitter # Dividir textos en textos mas chicos
from langchain.vectorstores import FAISS #Buscar en forma de vectores

from langchain_google_genai import ChatGoogleGenerativeAI 
from langchain.chains import RetrievalQA

# from langchain_community.embeddings import HuggingFaceInstructEmbeddings

from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
#Cargar y procesar el documento

loader = TextLoader('gastos.txt', encoding='utf-8')
documents = loader.load()

In [11]:
#Dividir el texto en chunks

text_splitter= CharacterTextSplitter(chunk_size=500, chunk_overlap=50) # Splitter recursivo
texts = text_splitter.split_documents(documents)

In [12]:
# Crear embeddings usando un modelo gratuito de HuggingFace

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_9788\2888338921.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
e:\Programacion\JD\Piragua\GeminiChat\venv310\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USUARIO\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WA

In [13]:
#Crear base de datos vectorial

db = FAISS.from_documents(texts, embeddings)

In [15]:
#Realizar una busuqeda

query = "Hola chat como estas? te pregunto ¿en que gasto mas mi dinero?."

# Inicializar el modelo de chat
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    google_api_key=os.getenv("GENAI_API_KEY"),
    temperature=0.7 #Creatividad del modelo
)

# Crear la cadena de recuperación y respuesta
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

# Ejecutar la consulta y obtener la respuesta
response = qa_chain.run(query)


# Reemplazar los asteriscos por negrita en Markdown
response = response.replace("* **", "**").replace("**:", "**").replace("*", "")

# Imprimir la respuesta
print(response)

Hola! Según la información proporcionada, gastas más dinero en el arriendo de tu apartamento, con un monto de 1.200.000 al mes.
